Ion simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PyonDetect.ion.ion import Ion, BerylliumIon, CalciumIon
from PyonDetect.estimator.estimator import EntropyGainEstimator, MinExpectedBinsEstimator, ThresholdingEstimator, AdaptiveMLEstimator, NonPoissonianAdaptiveMLEstimator
from PyonDetect.environment.environment import SimulationEnvironment

In [ ]:
# initialize simulation environment
R_D = 0.1 / 200 # per us
R_B = 25 / 200 #per us
initial_state = 1 # start in dark state
n_subbins = 30
t_detection = 200 # us

ion = Ion(initial_state, R_dark=R_D, R_bright=R_B)
estimator = MinExpectedBinsEstimator(R_D=R_D, R_B=R_B, e_c=1e-4, n_subbins_max=n_subbins, save_trajectory=True)
env = SimulationEnvironment([ion], [estimator], detection_time=t_detection, n_subbins=n_subbins, n_repetition=10000)

In [ ]:
# let simulation run
env.run()

In [ ]:
# save data to files
env.save_to_csv()
env.save_to_json()

In [ ]:
stats = env.get_stats()
traj = env.get_trajectories()
print(np.mean(stats['n_pi_pulses_0']))

In [ ]:
## some fun
e_c = 1e-4
ln_beta_ec = np.log((1 - e_c) / e_c)
R_B = 10 / 200 # per us
R_D = 1 / 200 # per us
delta_R = R_B - R_D

t_bin = 4
def alpha(R_B, t_bin):
    a = np.exp(-R_B * t_bin)
    coeff_alpha = 1 / (1 - a)
    return coeff_alpha

p_b = np.linspace(0.001, 0.999, 1000)
def decision(p_b, R_B, t_bin):
    return (1 - 2 * p_b) * (alpha(R_B, t_bin) - np.ceil((ln_beta_ec - np.log(p_b / (1-p_b))) / (delta_R * t_bin)))
plt.plot(p_b, decision(p_b, R_B, t_bin))
threshold_idx = np.argwhere(decision(p_b, R_B, t_bin) < 0)
print(threshold_idx)

In [ ]:
t_detection = 200
n_bins = np.arange(5, 30)
print(n_bins)
t_bin = t_detection / n_bins
print(t_bin)
plt.scatter(200/t_bin, t_bin * alpha(R_B, t_bin))
print(200/t_bin, t_bin * alpha(R_B, t_bin))

In [ ]:
# run test

def run_environment(n_bins, initial_state):
    R_D = 0 / 200 # per us
    R_B = 25 / 200 # per us
    e_c = 1e-4
    t_detection = 200
    ions = [Ion(initial_state, R_dark=R_D, R_bright=R_B) for i in range(3)]
    estimator1 = MinExpectedBinsEstimator(R_D=R_D, R_B=R_B, e_c=e_c, n_subbins_max=n_bins, save_trajectory=False)
    estimator2 = AdaptiveMLEstimator(R_D=R_D, R_B=R_B, e_c=e_c, n_subbins_max=n_bins, save_trajectory=False)
    estimator3 = EntropyGainEstimator(R_D=R_D, R_B=R_B, e_c=e_c, n_subbins_max=n_bins, n_counts_max=10, save_trajectory=False)
    estimators = [estimator1, estimator2, estimator3]
    env = SimulationEnvironment(ions, estimators, detection_time=t_detection, n_subbins=n_bins, n_repetition=100)
    env.run()
    env.save_to_csv()

In [ ]:
run_environment(10, 1)